## Speeding up Transformer training:

### Implementations:

1. English to French dataset

2. Automatic Mixed Precision

3. Dynamic padding (using `pad_sequence` and no manual pad count calculations)

4. One Cycle Policy

5. Parameter sharing

6. Scaled Dot Product Attention (SDP Kernel context, Flash option unsupported on T4 Colab)

7. Added code for Gradient Accumulation

8. Converted encoder_input and decoder_input Tensors in `_ _ getitem_ _` from `int64` to `int32` to reduce some memory (label in int32 gave error, need to look into loss function)

9. optimizer.zero_grad(set_to_none=True)

10. `.to(device, non_blocking=True)` (where applicable)

11. Was able to use `batch_size = 64 (batch size = 72 with 256 d_model)`


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [ ]:
rm -rf ERA1

In [ ]:
!git clone https://github.com/MANU-CHAUHAN/ERA1

Cloning into 'ERA1'...
remote: Enumerating objects: 557, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 557 (delta 40), reused 78 (delta 25), pack-reused 462
Receiving objects: 100% (557/557), 14.95 MiB | 13.99 MiB/s, done.
Resolving deltas: 100% (240/240), done.


In [ ]:
cd ERA1

/content/drive/My Drive/ERA1


In [ ]:
cd s16/

/content/drive/My Drive/ERA1/s16


In [ ]:
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.2 MB/s eta 0:00:00


In [ ]:
ls

config.py   model.py      READMEs16.md      runs/                      tokenizer_en.json  train.py
dataset.py  __pycache__/  requirements.txt  S16_optimized_final.ipynb  tokenizer_fr.json  weights/


In [ ]:
from config import get_config
import torch

cfg = get_config()

cfg

{'batch_size': 32,
 'num_epochs': 10,
 'lr': 0.001,
 'max_lr': 0.01,
 'pct_start': 0.1,
 'initial_div_factor': 10,
 'final_div_factor': 10,
 'anneal_strategy': 'linear',
 'three_phase': True,
 'seq_len': 500,
 'd_model': 512,
 'lang_src': 'en',
 'lang_tgt': 'fr',
 'model_folder': 'weights',
 'model_basename': 'tmodel_',
 'preload': False,
 'tokenizer_file': 'tokenizer_{0}.json',
 'experiment_name': 'runs/tmodel',
 'enable_amp': True,
 'd_ff': 512,
 'N': 6,
 'h': 8,
 'param_sharing': True,
 'gradient_accumulation': False,
 'accumulation_steps': 4}

In [ ]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

0

SDP Kernel: Math = True, Flash = False, Mem_Effciency = True

### with d_model = 256

#### cfg['gradient_accumulation'] = True

#### cfg['gradient_accumulation_steps'] = 40

In [ ]:
cfg['batch_size'] = 72
cfg['preload'] = False
cfg['num_epochs'] = 30
cfg['d_model'] = 256
cfg['d_ff'] = 128
cfg['pct_start'] = 0.2
cfg['max_lr'] = 10**-3
cfg['initial_div_factor'] = 10
cfg['final_div_factor'] = 10

cfg['gradient_accumulation'] = True
cfg['gradient_accumulation_steps'] = 40

from train import train_model

train_model(config=cfg)

👀Using device: cuda


Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

Max len of source sentence: 471
Max len of target sentence: 482

⚡️⚡️Number of model parameters: 25,824,850



Processing Epoch: 19:  34%|███▍      | 543/1589 [01:46<03:24,  5.12it/s, loss=1.72798, lr=[6.328636856273818e-05]]

In [ ]:
cfg['batch_size'] = 72
cfg['preload'] = True
cfg['num_epochs'] = 30
cfg['d_model'] = 256
cfg['d_ff'] = 128
cfg['pct_start'] = 0.2
cfg['max_lr'] = 10**-3
cfg['initial_div_factor'] = 10
cfg['final_div_factor'] = 10

cfg['gradient_accumulation'] = True
cfg['gradient_accumulation_steps'] = 40

from train import train_model

train_model(config=cfg)

👀Using device: cuda


Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

Max len of source sentence: 471
Max len of target sentence: 482

⚡️⚡️Number of model parameters: 25,824,850

Preloading model weights/tmodel_20.pt
Preloaded


Processing Epoch: 29: 100%|██████████| 1589/1589 [05:12<00:00,  5.09it/s, loss=1.65273, lr=[0.0005498583866568761]]


--------------------------------------------------------------------------------
    SOURCE: The cavalcade was brilliant, and its march resounded on the pavement.
    TARGET: La cavalcade était brillante et résonnait sur le pavé.
 PREDICTED: La cavalcade était brillante et sa marche s ' agitait sur le pavé .
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
    SOURCE: There was nothing extraordinary about the country; the sky was blue, the trees swayed; a flock of sheep passed.
    TARGET: Mais non! la campagne n’avait rien d’extraordinaire: le ciel était bleu, les arbres se balançaient; un troupeau de moutons passa.
 PREDICTED: Le ciel était bleu , des arbres , des cris de mouflons , un troupeau de mouflons passa .
--------------------------------------------------------------------------------


RuntimeError: ignored